1. Add models for student, subject and student_subject in SQLAlchemy.

```sql
CREATE TABLE student (
    id serial PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    age INT NOT NULL
);
```

```sql
CREATE TABLE subject (
    id serial PRIMARY KEY,
    name VARCHAR(50) NOT NULL
);
```

```sql
CREATE TABLE student_subject (
    id serial PRIMARY KEY,
    student_id INT NOT NULL,
    subject_id INT NOT NULL,

    FOREIGN KEY (student_id)
        REFERENCES student (id),
    FOREIGN KEY (subject_id)
        REFERENCES subject (id)
);
```

In [23]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Student(Base):
    __tablename__ = 'student'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    age = Column(Integer, nullable=False)
    
    subjects = relationship('Subject', secondary='student_subject', back_populates='students')

class Subject(Base):
    __tablename__ = 'subject'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)

    students = relationship('Student', secondary='student_subject', back_populates='subjects')

class StudentSubject(Base):
    __tablename__ = 'student_subject'

    id = Column(Integer, primary_key=True)
    student_id = Column(Integer, ForeignKey('student.id'), nullable=False)
    subject_id = Column(Integer, ForeignKey('subject.id'), nullable=False)

engine = create_engine('sqlite:///database.db')
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


2. Find all students` name that visited 'English' classes.

In [25]:
# Create the database engine and session
engine = create_engine("sqlite:///database.db")
Session = sessionmaker(bind=engine)
session = Session()

# Query to find students' names who visited 'English' classes
english_students = session.query(Student.name).join(StudentSubject).join(Subject).filter(Subject.name == 'English').all()

# Print the results
for student_name in english_students:
    print(student_name[0])


3. (Optional): Rewrite queries from the previous lesson using SQLAlchemy.

INSERT Queries

1. Insert one row:

In [26]:
new_student = Student(name='Bae', age=18)
session.add(new_student)
session.commit()

2. Insert multiple rows:

In [27]:
students_to_insert = [
    Student(name='Bae', age=18),
    Student(name='Eddy', age=21),
    Student(name='Lily', age=22),
    Student(name='Jenny', age=19)
]

session.add_all(students_to_insert)
session.commit()

3. With column names:

In [ ]:
new_students = [
    Student(name='Bae', age=18),
    Student(name='Eddy', age=21),
    Student(name='Lily', age=22),
    Student(name='Jenny', age=19)
]

session.bulk_save_objects(new_students)
session.commit()

SELECT Queries:

1. Select all rows:

In [31]:
all_students = session.query(Student).all()

2. Select specific columns:

In [29]:
student_names = session.query(Student.name).all()

WHERE Clause:

1. Select with WHERE:

In [32]:
young_students = session.query(Student).filter(Student.age < 20).all()

2. AND and OR conditions:

In [ ]:
students_filtered = session.query(Student).filter(Student.age < 20, Student.age > 10).all()
students_age_name = session.query(Student).filter((Student.age < 20) | (Student.name == 'John')).all()

ORDER BY Clause:

1. Order by age:

In [34]:
students_ordered_by_age = session.query(Student).order_by(Student.age).all()

2. Order by age in descending order with filtering:

In [35]:
students_filtered_ordered = session.query(Student).filter(Student.age == 20).order_by(Student.age.desc()).all()

INNER JOIN:

1. Simple join between two tables:

In [37]:
from sqlalchemy.orm import aliased

StudentSubject = aliased(StudentSubject)

result = session.query(Student, StudentSubject).join(StudentSubject, Student.id == StudentSubject.student_id).all()

2. Select columns during the join:

In [38]:
result = session.query(Student.name, StudentSubject.subject_id).join(StudentSubject, Student.id == StudentSubject.student_id).all()

3. Join with three tables:

In [39]:
Subject = aliased(Subject)
result = session.query(Student, StudentSubject, Subject).join(StudentSubject, Student.id == StudentSubject.student_id).join(Subject, Subject.id == StudentSubject.subject_id).all()

4. Subquery join:

In [40]:
subquery = session.query(Student.id, Subject.id).join(Subject, Subject.id == Student.id).subquery()
result = session.query(subquery).all()

FULL JOIN:

In [ ]:
from sqlalchemy.sql import text

query = text("SELECT * FROM student FULL JOIN student_subject ON student.id = student_subject.student_id;")
result = session.execute(query).fetchall()


LEFT/RIGHT JOIN:

1. Left join:

In [41]:
result = session.query(Student, StudentSubject).outerjoin(StudentSubject, Student.id == StudentSubject.student_id).all()

2. Right join:

In [42]:
result = session.query(Student, StudentSubject).outerjoin(StudentSubject, Student.id == StudentSubject.student_id, isouter=True).all()

AGGREGATE FUNCTIONS:

1. Count:

In [43]:
from sqlalchemy import func

count = session.query(func.count()).select_from(Student).scalar()

2. AVG:

In [44]:
average_age = session.query(func.avg(Student.age)).scalar()

3. MAX:

In [45]:
max_age = session.query(func.max(Student.age)).scalar()

4. MIN:

In [ ]:
min_age = session.query(func.min(Student.age)).scalar()

5. SUM:

In [47]:
total_age = session.query(func.sum(Student.age)).scalar()

UPDATE:

Update one column in a row:

In [48]:
student_to_update = session.query(Student).filter_by(name='Bae').first()
if student_to_update:
    student_to_update.age = 20
    session.commit()

DELETE:

1. Delete one or multiple rows based on a condition:

In [49]:
students_to_delete = session.query(Student).filter_by(name='Bae').all()
for student in students_to_delete:
    session.delete(student)
session.commit()

2. Delete all rows from the table:

In [ ]:
session.query(Student).delete()
session.commit()